In [ ]:
!pip install numpy requests nlpaug
!pip install transformers
!pip install sacremoses


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=ee826f4ce22e7dc38533ed30f10ba06ea297e9d8bfff396115d43c9b305dac5f
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [ ]:
import pandas as pd
import requests
import shutil
import os
import gdown

import gzip
import re
import string
import nlpaug

In [ ]:
def load_and_preprocess_data(file_path):
  df = pd.read_csv(file_path, names=['Emotion', 'Text', 'DNTKNOW']).drop(columns=['DNTKNOW']).dropna()
  df['Text_processed'] = df.Text.apply(clean_text)
  return df

def clean_text(text):
  # to lower case
  text = text.lower()
  # remove links
  text = re.sub('https:\/\/\S+', '', text)
  # remove punctuation
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  # remove next line
  text = re.sub(r'[^ \w\.]', '', text)
  # remove words containing numbers
  text = re.sub('\w*\d\w*', '', text)

  return text


In [ ]:
text = 'The quick brown fox jumps over the lazy dog .'
print(text)

The quick brown fox jumps over the lazy dog .


Substitute word by word2vec similarity

### FIRST INSTALLING DEPENDENCIES AND MODELS

In [ ]:

# Especifica el nombre de la carpeta a crear
folder_name = "models"

# Crea la carpeta si no existe
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


url = 'https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM'
output = 'GoogleNews-vectors-negative300.bin.gz'

gdown.download(url, output, quiet=False)

input_file = 'GoogleNews-vectors-negative300.bin.gz'
output_file = 'GoogleNews-vectors-negative300.bin'

with gzip.open(input_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        f_out.write(f_in.read())


# Especifica la ruta de la carpeta de destino
destination_folder = 'models'

# Mueve el archivo descomprimido a la carpeta de destino
shutil.move('GoogleNews-vectors-negative300.bin', f'{destination_folder}/GoogleNews-vectors-negative300.bin')



Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
100%|██████████| 1.65G/1.65G [00:27<00:00, 59.7MB/s]


'models/GoogleNews-vectors-negative300.bin'

## Word2vec

In [ ]:
model_dir = "/models/"
aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='/content/models/GoogleNews-vectors-negative300.bin',
    action="insert")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

In [ ]:
# model_type: word2vec, glove or fasttext
aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='/content/models/GoogleNews-vectors-negative300.bin',
    action="substitute")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

## TRANSFORMERS SUBSTITUTION AND INSERTION

In [ ]:
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

In [ ]:
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

## BACK TRANSLATION

In [ ]:
import nlpaug.augmenter.word as naw

text = 'The quick brown fox jumped over the lazy dog'
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de',
    to_model_name='facebook/wmt19-de-en'
)
back_translation_aug.augment(text)

In [ ]:

url = "https://raw.githubusercontent.com/PoorvaRane/Emotion-Detector/master/ISEAR.csv"
output_file = "ISEAR.csv"
destination_folder = "data"

response = requests.get(url)
with open(output_file, 'wb') as f:
    f.write(response.content)

shutil.move(output_file, f"{destination_folder}/{output_file}")

# Load and preprocess the dataset
df = load_and_preprocess_data('/content/data/ISEAR.csv')
df['Emotion'] = df['Emotion'].replace('guit', 'guilt')

In [ ]:
import requests
import shutil
import pandas as pd
import nlpaug.augmenter.word as naw
import numpy as np
from sklearn.utils import shuffle

# Descarga y procesa el csv
url = "https://raw.githubusercontent.com/PoorvaRane/Emotion-Detector/master/ISEAR.csv"
output_file = "ISEAR.csv"
destination_folder = "data"

response = requests.get(url)
with open(output_file, 'wb') as f:
    f.write(response.content)

shutil.move(output_file, f"{destination_folder}/{output_file}")

# Carga y preprocesa el dataset
df = load_and_preprocess_data(f'{destination_folder}/{output_file}')
df['Emotion'] = df['Emotion'].replace('guit', 'guilt')

# Divide el df en cinco partes iguales
df_split = np.array_split(df, 5)

# Guarda cada fragmento en un archivo separado
for i in range(5):
    df_split[i].to_csv(f'df_part_{i}.csv')


NameError: ignored

In [ ]:
# Para el primer fragmento
augmenter = naw.WordEmbsAug(
    model_type='word2vec', model_path='/content/models/GoogleNews-vectors-negative300.bin',
    action="substitute")
df = pd.read_csv('df_part_0.csv')
df['Augmented'] = df['Text_processed'].apply(lambda x: augmenter.augment(x))
df.to_csv('df_part_0.csv')


In [ ]:
# Para el segundo fragmento
augmenter = naw.WordEmbsAug(
    model_type='word2vec', model_path='/content/models/GoogleNews-vectors-negative300.bin',
    action="insert")
df = pd.read_csv('df_part_1.csv')
df['Augmented'] = df['Text_processed'].apply(lambda x: augmenter.augment(x))


In [ ]:
# Para el tercer fragmento
augmenter = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")
df = pd.read_csv('df_part_2.csv')
df['Augmented'] = df['Text_processed'].apply(lambda x: augmenter.augment(x))
df.to_csv('df_part_2.csv')


In [ ]:
# Para el cuarto fragmento
augmenter = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute")
df = pd.read_csv('df_part_3.csv')
df['Augmented'] = df['Text_processed'].apply(lambda x: augmenter.augment(x))
df.to_csv('df_part_3.csv')


In [ ]:
# Para el quinto fragmento
augmenter = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de',
    to_model_name='facebook/wmt19-de-en')
df = pd.read_csv('df_part_4.csv')
df['Augmented'] = df['Text_processed'].apply(lambda x: augmenter.augment(x))


In [ ]:
# Combina los fragmentos de nuevo en un solo df
new_df = pd.concat([pd.read_csv(f'df_part_{i}.csv') for i in range(5)])

# Haz shuffle de los datos para asegurar una buena mezcla
new_df = shuffle(new_df)
